In [14]:
import pandas as pd
import pickle
import nltk
from gensim import corpora, models
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore")

In [2]:
df_vec=pd.read_pickle('assets/df_vec.pkl')

In [3]:
# code help from J's lesson LDA_in_class
texts = [df_vec.columns[df_vec.loc[index,:].to_numpy().nonzero()] for index in df_vec.index]
dictionary = corpora.Dictionary(texts)      
corpus = [dictionary.doc2bow(text) for text in texts] 

In [4]:
pickle.dump(corpus, open('assets/corpus.pkl', 'wb'))
dictionary.save('assets/dictionary.gensim')

In [5]:
texts[0]

Index(['cookson eisenack', 'dinoflagel cyst', 'geolog societi',
       'geolog survei', 'lentin william', 'lower cretac', 'north sea',
       'shown figur', 'societi london', 'stover evitt'],
      dtype='object')

# Latent Dirichlet Allocation

In [6]:
lda_model = models.ldamodel.LdaModel(corpus,
                                            num_topics=6,
                                            id2word=dictionary,
                                            passes=20,
                                           eval_every = 5,
                                           decay = .4,
                                           offset = 60.0,
                                            minimum_probability = 0.05
                                           )
lda_model.save('assets/gensim_lda_model.gensim')

## Taking a closer look at the topics

In [8]:
topics = lda_model.print_topics(num_words=3)
for topic in topics:
    print(topic)

(0, '0.031*"deep sea" + 0.027*"earli eocen" + 0.027*"paleocen eocen"')
(1, '0.037*"geolog societi" + 0.029*"geolog survei" + 0.028*"special public"')
(2, '0.023*"gulf coast" + 0.020*"geolog societi" + 0.019*"coastal plain"')
(3, '0.049*"spore pollen" + 0.048*"pollen spore" + 0.042*"dinoflagel cyst"')
(4, '0.027*"sedimentari structur" + 0.025*"grain size" + 0.024*"new york"')
(5, '0.070*"palaeoclimatolog palaeoecolog" + 0.068*"palaeogeographi palaeoclimatolog" + 0.046*"geol soc"')


## Scoring

In [9]:
from gensim.models.coherencemodel import CoherenceModel

coherence_model_lda = CoherenceModel(model=lda_model,
                                     texts=texts,
                                     dictionary=dictionary,
                                     coherence='c_v')

coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)   # higher is better
print('Perplexity:', lda_model.log_perplexity(corpus))  #lower is better

Coherence Score:  0.7529275071597289
Perplexity: -4.9752722579859


## Visualize LDA model in pyLDAvis

In [15]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.184963 -0.121953       1        1  26.191599
4     -0.046888 -0.126111       2        1  24.534498
1     -0.095106  0.061699       3        1  15.630860
0     -0.008579  0.092932       4        1  13.819485
5      0.289447 -0.107294       5        1  12.657459
3      0.046089  0.200727       6        1   7.166097, topic_info=    Category        Freq                              Term       Total  \
10   Default  439.000000     palaeoclimatolog palaeoecolog  439.000000   
85   Default  427.000000  palaeogeographi palaeoclimatolog  427.000000   
15   Default  171.000000                          http doi  171.000000   
14   Default  168.000000                           doi org  168.000000   
42   Default  198.000000                      pollen spore  198.000000   
12   Default  175.000000                      spore pollen  175.000000   
84   Default  296.000000                           fig fig  296.000000   
1    Default  185.000000                   dinoflagel cyst  185.000000   
94   Default  419.000000                          geol soc  419.000000   
105  Default  178.000000        palaeogeogr palaeoclimatol  178.000000   
104  Default  186.000000         palaeoclimatol palaeoecol  186.000000   
138  Default  139.000000                      pollen grain  139.000000   
65   Default  333.000000                   springer verlag  333.000000   
16   Default  129.000000                  sepm specialpubl  129.000000   
51   Default  189.000000                          http www  189.000000   
186  Default  218.000000              sedimentari structur  218.000000   
93   Default  108.000000                   earthplanet sci  108.000000   
96   Default  126.000000                          sci lett  126.000000   
4    Default   92.000000                    lentin william   92.000000   
3    Default  395.000000                     geolog survei  395.000000   
102  Default  217.000000                         spec publ  217.000000   
0    Default   77.000000                  cookson eisenack   77.000000   
119  Default  213.000000                       earli eocen  213.000000   
110  Default  218.000000                          deep sea  218.000000   
26   Default  259.000000                       late cretac  259.000000   
9    Default   74.000000                      stover evitt   74.000000   
38   Default  307.000000                      upper cretac  307.000000   
2    Default  442.000000                    geolog societi  442.000000   
95   Default  124.000000                         geophi re  124.000000   
89   Default   75.000000                          text fig   75.000000   
..       ...         ...                               ...         ...   
89    Topic6   63.260754                          text fig   75.561073   
86    Topic6   60.899059                         plate fig   73.541550   
87    Topic6   46.074596                       plate figur   58.132645   
142   Topic6   37.317169                     thomson pflug   56.891273   
12    Topic6  106.174469                      spore pollen  175.386688   
138   Topic6   80.643425                      pollen grain  139.848907   
42    Topic6  104.207016                      pollen spore  198.443115   
1     Topic6   92.511765                   dinoflagel cyst  185.424591   
41    Topic6   55.371689                       new zealand  157.192780   
84    Topic6   88.690094                           fig fig  296.497559   
5     Topic6   52.434174                      lower cretac  175.622742   
193   Topic6   29.639917                         scale bar  103.591759   
141   Topic6   40.356953                         geol surv  158.689529   
88    Topic6   21.693474                           sam ple   85.804970   
139   Topic6   28.969801                         rel abund  131.691727   
26    Topic6   50.086704                       

In [16]:
#save viz to output file
pyLDAvis.save_html((pyLDAvis.gensim.prepare(lda_model, corpus, dictionary)), 'pyldavis_output.html' )